In [19]:
from ncclient import manager
from xml.dom import minidom
import xml.etree.ElementTree as ET

In [20]:
HOST = 'sandbox-iosxr-1.cisco.com'
PORT = 830  # NETCONF Port
USERNAME = 'admin'
PASSWORD = 'C1sco12345'

In [21]:
# Crear una sesión NETCONF
with manager.connect(
    host=HOST,
    port=PORT,
    username=USERNAME,
    password=PASSWORD,
    hostkey_verify=False,  # Para evitar problemas con certificados en el sandbox
    device_params={'name': 'csr'}
) as m:
    # Realizar una solicitud GET con el filtro XPath para obtener la tabla de enrutamiento IPv4
    netconf_reply = m.get_config(source='running')

In [22]:
# Transforma la respuesta obtenida para que sea un string en formato xml
stringReply = minidom.parseString(netconf_reply.xml).toprettyxml()

In [24]:
# Cambia el string para que sea un arbol, de manera que se pueda realizar una busqueda por etiqueta en python
root = ET.fromstring(stringReply) 

In [26]:
routing_table = {}

# Busca los atributos pertenencientes a la tabla de enrutamiento
for interface_config in root.findall('.//{http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg}interface-configuration'):
    interface_name = interface_config.find('{http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg}interface-name').text
    ipv4_network = interface_config.find('.//{http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-io-cfg}ipv4-network')

    # Descarta los elementos que no tienen asignados una direccion ip
    if ipv4_network is not None:
        address = ipv4_network.find('.//{http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-io-cfg}address').text
        netmask = ipv4_network.find('.//{http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-io-cfg}netmask').text
        routing_table[interface_name] = {'address': address, 'netmask': netmask}

# Imprimir la tabla de enrutamiento
for interface, data in routing_table.items():
    print(f"Interface: {interface}, Address: {data['address']}, Netmask: {data['netmask']}")

Interface: Loopback100, Address: 1.1.1.100, Netmask: 255.255.255.255
Interface: Loopback555, Address: 3.3.3.6, Netmask: 255.255.255.255
Interface: MgmtEth0/RP0/CPU0/0, Address: 10.10.20.175, Netmask: 255.255.255.0
Interface: MgmtEth1/RP0/CPU0/0, Address: 2.2.2.2, Netmask: 255.255.255.255
